In [1]:
from artifacts_ipv6_sra_scanning.config import *
import pickle
import os
import ipaddress
import glob
from tqdm.auto import tqdm
import country_converter as coco
from parallel_pandas import ParallelPandas
ParallelPandas.initialize(n_cpu=24, split_factor=1, disable_pr_bar=False)

2025-10-06 14:25:07.336 | INFO     | artifacts_ipv6_sra_scanning.config:<module>:22 - PROJ_ROOT path is: /home/service/artifacts-ipv6-sra-scanning


In [2]:
df_bgp_sra_as = pl.read_csv(f'{INTERIM_DATA_DIR}/init-ips-sra-bgp48-scan.csv')

In [3]:
def load_and_join(files):
    df = pl.DataFrame()
    for i in tqdm(range(len(files))):
        if df.is_empty():
            df = pl.read_csv(files[i],columns=['initial-ip','saddr']).unique(subset=['initial-ip'])
        else:
            tmp = pl.read_csv(files[i],columns=['initial-ip','saddr']).unique(subset=['initial-ip'])
            df = df.join(tmp,on='initial-ip',how='left', suffix=f'_s{i}')
    return df

In [4]:
file1 = '/home/service/ipv6-measurements-mkoch/backup_2024/sra-analysis/data/bgp48_sra_checks/zmap_icmp_bgp_48_sra_1728075185.log.zst'

In [5]:
df_bgp_reply = load_and_join([file1])

  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
df_bgp_reply.head()

initial-ip,saddr
str,str
"""2a0c:ea47:ec7e::""","""2a0c:ea40::1"""
"""2a03:7f24:25fd::""","""2a02:7e00:5:e::2"""
"""2a13:3c07:6286::""","""2a00:6340:2040:3422::2"""
"""2a03:3cc0:4537::""","""2a02:28:1:2::6c:22"""
"""2a13:d9c0:b5e7::""","""2001:1a98:6677::9d9d:3961"""


In [7]:
df_router_ips = pl.read_csv(f'{INTERIM_DATA_DIR}/router-ips.csv')

In [8]:
df_router_ips.head()

ip-addr,Geo,AS-Number,BGP-Prefix,Org
str,str,f64,str,str
"""2402:b400:4440:53ef:c822:8bff:…","""MYS""",45960.0,"""2402:b400:4440::/46""","""YTLCOMMS-AS-AP YTL COMMUNICATI…"
"""2003:e6:37ff:3ab2:3ea6:2fff:fe…","""DEU""",3320.0,"""2003::/19""","""DTAG Internet service provider…"
"""2405:8d40:4081:579:1807:c46a:e…","""PHL""",139831.0,"""2405:8d40:4080::/42""","""DTC-AS-AP DITO TELECOMMUNITY C…"
"""240e:358:9c0a:13ec:ae64:62ff:f…","""CHN""",4134.0,"""240e:358::/29""","""CHINANET-BACKBONE No.31,Jin-ro…"
"""2408:8441:5808:10fa:1807:f6b5:…","""CHN""",4837.0,"""2408:8441::/32""","""CHINA169-BACKBONE CHINA UNICOM…"


In [9]:
df_bgp_sra_as

initial-ip,Geo,AS-Number,BGP-Prefix,Org
str,str,f64,str,str
"""2a0c:885:7d93::""","""UKR""",200000.0,"""2a0c:880::/29""","""UKRAINE-AS"""
"""2804:7b68:7d35::""","""BRA""",271430.0,"""2804:7b68::/32""","""SINAL NET FIBRA"""
"""2a14:0:469d::""","""DEU""",216176.0,"""2a14::/32""","""AS216176"""
"""2804:6e4:7402::""","""BRA""",262612.0,"""2804:6e4::/33""","""Piotr Piwowar"""
"""2604:14c0:7603::""","""USA""",22121.0,"""2604:14c0::/32""","""TOPSHAM-COMMUNICATIONS"""
…,…,…,…,…
"""2804:7c74:b906::""","""BRA""",271497.0,"""2804:7c74::/32""","""Confiber Telcomunicacoes ltda"""
"""2407:a080:1f4e::""","""AUS""",133480.0,"""2407:a080::/32""","""INT-5GN-AS-AP 5G NETWORK OPERA…"
"""2a02:d5f:d896::""","""CHE""",44919.0,"""2a02:d58::/29""","""EQIPE"""


In [10]:
tmp = df_bgp_sra_as.join(df_bgp_reply,how='left',on='initial-ip')

In [11]:
tmp

initial-ip,Geo,AS-Number,BGP-Prefix,Org,saddr
str,str,f64,str,str,str
"""2a0c:885:7d93::""","""UKR""",200000.0,"""2a0c:880::/29""","""UKRAINE-AS""","""2001:67c:2070:b901::b"""
"""2804:7b68:7d35::""","""BRA""",271430.0,"""2804:7b68::/32""","""SINAL NET FIBRA""","""2804:c84:10:101::2e"""
"""2a14:0:469d::""","""DEU""",216176.0,"""2a14::/32""","""AS216176""","""2001:19f0:6c00:18b2:5400:4ff:f…"
"""2804:6e4:7402::""","""BRA""",262612.0,"""2804:6e4::/33""","""Piotr Piwowar""","""2804:6e4:1::5"""
"""2604:14c0:7603::""","""USA""",22121.0,"""2604:14c0::/32""","""TOPSHAM-COMMUNICATIONS""","""2607:ffc8:0:6::10"""
…,…,…,…,…,…
"""2804:7c74:b906::""","""BRA""",271497.0,"""2804:7c74::/32""","""Confiber Telcomunicacoes ltda""","""2804:40b0:7::67d"""
"""2407:a080:1f4e::""","""AUS""",133480.0,"""2407:a080::/32""","""INT-5GN-AS-AP 5G NETWORK OPERA…","""2407:a080::1a"""
"""2a02:d5f:d896::""","""CHE""",44919.0,"""2a02:d58::/29""","""EQIPE""","""2a02:d58:1:58::2"""


In [12]:
tmp = tmp.join(df_router_ips,how='left',left_on='saddr',right_on='ip-addr',suffix='_saddr')

In [13]:
tmp

initial-ip,Geo,AS-Number,BGP-Prefix,Org,saddr,Geo_saddr,AS-Number_saddr,BGP-Prefix_saddr,Org_saddr
str,str,f64,str,str,str,str,f64,str,str
"""2a0c:885:7d93::""","""UKR""",200000.0,"""2a0c:880::/29""","""UKRAINE-AS""","""2001:67c:2070:b901::b""","""UKR""",200000.0,"""2001:67c:2070::/48""","""UKRAINE-AS"""
"""2804:7b68:7d35::""","""BRA""",271430.0,"""2804:7b68::/32""","""SINAL NET FIBRA""","""2804:c84:10:101::2e""","""BRA""",52720.0,"""2804:c84::/32""","""WEBFOCO TELECOMUNICACOES LTDA"""
"""2a14:0:469d::""","""DEU""",216176.0,"""2a14::/32""","""AS216176""","""2001:19f0:6c00:18b2:5400:4ff:f…","""DEU""",20473.0,"""2001:19f0:6c00::/38""","""AS-VULTR"""
"""2804:6e4:7402::""","""BRA""",262612.0,"""2804:6e4::/33""","""Piotr Piwowar""","""2804:6e4:1::5""","""BRA""",262612.0,"""2804:6e4::/33""","""Piotr Piwowar"""
"""2604:14c0:7603::""","""USA""",22121.0,"""2604:14c0::/32""","""TOPSHAM-COMMUNICATIONS""","""2607:ffc8:0:6::10""","""USA""",17356.0,"""2607:ffc8::/32""","""VERMONT-TELE"""
…,…,…,…,…,…,…,…,…,…
"""2804:7c74:b906::""","""BRA""",271497.0,"""2804:7c74::/32""","""Confiber Telcomunicacoes ltda""","""2804:40b0:7::67d""","""BRA""",265975.0,"""2804:40b0::/32""","""INTEREDE TELECOM"""
"""2407:a080:1f4e::""","""AUS""",133480.0,"""2407:a080::/32""","""INT-5GN-AS-AP 5G NETWORK OPERA…","""2407:a080::1a""","""AUS""",133480.0,"""2407:a080::/48""","""INT-5GN-AS-AP 5G NETWORK OPERA…"
"""2a02:d5f:d896::""","""CHE""",44919.0,"""2a02:d58::/29""","""EQIPE""","""2a02:d58:1:58::2""","""CHE""",44919.0,"""2a02:d58::/29""","""EQIPE"""


In [14]:
tmp.select('AS-Number').n_unique()

17652

In [15]:
tmp.select('AS-Number_saddr').n_unique()

12284

In [16]:
tmp.filter(pl.col('AS-Number')!=pl.col('AS-Number_saddr'))

initial-ip,Geo,AS-Number,BGP-Prefix,Org,saddr,Geo_saddr,AS-Number_saddr,BGP-Prefix_saddr,Org_saddr
str,str,f64,str,str,str,str,f64,str,str
"""2804:7b68:7d35::""","""BRA""",271430.0,"""2804:7b68::/32""","""SINAL NET FIBRA""","""2804:c84:10:101::2e""","""BRA""",52720.0,"""2804:c84::/32""","""WEBFOCO TELECOMUNICACOES LTDA"""
"""2a14:0:469d::""","""DEU""",216176.0,"""2a14::/32""","""AS216176""","""2001:19f0:6c00:18b2:5400:4ff:f…","""DEU""",20473.0,"""2001:19f0:6c00::/38""","""AS-VULTR"""
"""2604:14c0:7603::""","""USA""",22121.0,"""2604:14c0::/32""","""TOPSHAM-COMMUNICATIONS""","""2607:ffc8:0:6::10""","""USA""",17356.0,"""2607:ffc8::/32""","""VERMONT-TELE"""
"""2a04:1600:a3ab::""","""DEU""",61967.0,"""2a04:1600::/29""","""AS-TELEMATIS *** SkyLink XSSe…","""2001:4d50:f016::c:1a""","""DEU""",34309.0,"""2001:4d50::/32""","""LINK11 Link11 GmbH"""
"""2a01:5900:7e55::""","""DEU""",29239.0,"""2a01:5900::/29""","""I-NETPARTNER-AS""","""2a00::ffc:0:e00:2:12""","""DEU""",3209.0,"""2a00::/22""","""VODANET International IP-Backb…"
…,…,…,…,…,…,…,…,…,…
"""2a06:d984:a988::""","""IRQ""",51539.0,"""2a06:d980::/29""","""ASN-AUIS-NET""","""2a02:b60:1ffe:0:c953::2""","""IRQ""",44217.0,"""2a02:b60::/32""","""IQNETWORKS"""
"""2804:3054:6990::""","""BRA""",264944.0,"""2804:3054::/32""","""enzo servicos e comercio de te…","""2804:2480:0:1::6""","""BRA""",264227.0,"""2804:2480::/32""","""INOVE COMERCIO E SERVICOS DE T…"
"""2a0c:502:3127::""","""DEU""",205103.0,"""2a0c:500::/30""","""TELEGRAPH-AS""","""2001:7f8::3:212f:0:1""","""DEU""",29632.0,"""2001:7f8::/64""","""NASSIST-AS"""


In [17]:
tmp.write_parquet(f'{PROCESSED_DATA_DIR}/bgp-sra-AS-comparison.parquet')